In [11]:
import random
import re
from abc import ABC, abstractmethod
import logging

import numpy as np
from scipy.optimize import differential_evolution

import cv2
from matplotlib import pyplot as plt


Определяем функцию MSE

In [12]:
def mse(image1, image2):
    return np.mean((image1 - image2) ** 2)

Оптимизаторы

In [ ]:
class Optimizer(ABC):
    @abstractmethod
    def optimize(self, template_image, data_images, r_min, r_max, max_iterations, precision):
        pass

class PeanoOptimizer(Optimizer):
    def optimize(self, template_image, data_images, r_min, r_max, max_iterations, precision):
        bounds = [(r_min, r_max)]
        result = differential_evolution(
            lambda r: mse(template_image, self.generate_image_with_radius(data_images, r)),
            bounds,
            maxiter=max_iterations,
            disp=False
        )
        return result.x[0], result.fun

    def generate_image_with_radius(self, data_images, radius):
        return data_images[0]
    
class MonteCarloOptimizer(Optimizer):
    def optimize(self, template_image, data_images, r_min, r_max, max_iterations, precision):
        best_mse = float('inf')
        best_radius = None
        
        for _ in range(max_iterations):
            r = random.uniform(r_min, r_max)
            current_mse = mse(template_image, self.generate_image_with_radius(data_images, r))
            
            if current_mse < best_mse:
                best_mse = current_mse
                best_radius = r
                
            if best_mse <= precision:
                break
                
        return best_radius, best_mse

    def generate_image_with_radius(self, data_images, radius):
        return data_images[0]
    
class GridSearchOptimizer(Optimizer):
    def optimize(self, template_image, data_images, r_min, r_max, max_iterations, precision):
        radii = np.linspace(r_min, r_max, max_iterations)
        best_mse = float('inf')
        best_radius = None
        
        for r in radii:
            current_mse = mse(template_image, self.generate_image_with_radius(data_images, r))
            
            if current_mse < best_mse:
                best_mse = current_mse
                best_radius = r
                
            if best_mse <= precision:
                break
                
        return best_radius, best_mse

    def generate_image_with_radius(self, data_images, radius):
        return data_images[0] 
     

Менеджер расчетов

In [14]:
class CalculationManager:
    def __init__(self, log_file='calculations.log'):
        logging.basicConfig(filename=log_file, level=logging.INFO)

    def log_calculation(self, radius, mse_value):
        logging.info(f'Radius: {radius}, MSE: {mse_value}')
        
    def run_calculations(self, template_image, data_images, optimizer_class, r_min, r_max, max_iterations=1000, precision=1e-6):
        optimizer = optimizer_class()
        radius, mse_value = optimizer.optimize(template_image, data_images, r_min, r_max, max_iterations, precision)
        self.log_calculation(radius, mse_value)
        return radius, mse_value

In [ ]:
# Загрузка изображений 
template_image = np.random.rand(100, 100)
# template_image = cv2.imread('cell_patch/0/0.png', cv2.IMREAD_GRAYSCALE)
data_images = [np.random.rand(100, 100) for _ in range(10)]  # Пример данных
# data_images = [cv2.imread('generated/6.png', cv2.IMREAD_GRAYSCALE)]

# Параметры
r_min = 5
r_max = 20
max_iterations = 100
precision = 1e-6
# Менеджер расчетов
manager = CalculationManager()
# Оптимизация с использованием разных методов
for optimizer_class in [PeanoOptimizer, MonteCarloOptimizer, GridSearchOptimizer]:
    print(f'Running optimization with {optimizer_class.__name__}')
    radius, mse_value = manager.run_calculations(template_image, data_images,
                                                  optimizer_class,
                                                  r_min, r_max,
                                                  max_iterations,
                                                  precision)
    print(f'Best radius: {radius}, Best MSE: {mse_value}')

Running optimization with PeanoOptimizer
Best radius: 18.59589165575664, Best MSE: 0.1663576404942593
Running optimization with MonteCarloOptimizer
Best radius: 14.31625341027811, Best MSE: 0.1663576404942593
Running optimization with GridSearchOptimizer
Best radius: 5.0, Best MSE: 0.1663576404942593
